In [3]:
!gdown 1cfUt8-cJR7gWLv68ya_gZwbIZgq29F7D
!unzip 1mcorpus.zip

Downloading...
From: https://drive.google.com/uc?id=1cfUt8-cJR7gWLv68ya_gZwbIZgq29F7D
To: /content/1mcorpus.zip
100% 122M/122M [00:01<00:00, 110MB/s]
Archive:  1mcorpus.zip
   creating: 1mcorpus/
  inflating: 1mcorpus/corpus.en_ru.1m.en  
  inflating: 1mcorpus/corpus.en_ru.1m.ru  


In [50]:
!pip install razdel
!pip install iopath

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 895.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=8072673383d540f0f35ddb3c7878159e82676645088dc6524fb286fc5b158bec
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath


In [104]:
import torch
import torchdata
import torchtext
import re
from razdel import tokenize as rus_tokenizer
from nltk.tokenize import word_tokenize as eng_tokenizer
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [97]:
def print_simple_data_pipe(date_pipe: torchdata.datapipes.iter.IterDataPipe, n=10) -> None:
  print(type(date_pipe))
  x = 0
  for sample in date_pipe:
    print(sample)
    if x == n:
      break
    x +=1


In [98]:
FOLDER = './1mcorpus/'

datapipe_en = torchdata.datapipes.iter.FileLister([FOLDER]).filter(lambda fname: fname.endswith('.en'))
datapipe_rus = torchdata.datapipes.iter.FileLister([FOLDER]).filter(lambda fname: fname.endswith('.ru'))

datapipe_en = torchdata.datapipes.iter.FileOpener(datapipe_en, mode='rt', length=40)
datapipe_rus = torchdata.datapipes.iter.FileOpener(datapipe_rus, mode='rt', length=40)

datapipe_en = torchdata.datapipes.iter.IterableWrapper(datapipe_en.readlines(return_path=False))
datapipe_rus = torchdata.datapipes.iter.IterableWrapper(datapipe_rus.readlines(return_path=False))

datapipe_pairs = torchdata.datapipes.iter.Zipper(datapipe_en, datapipe_rus)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/utils/common.py:149: UserWarning: Lambda function is not supported by pickle, please use regular python function or functools.partial instead.
  warnings.warn(


In [112]:
[i.text for i in rus_tokenizer('Привет мир')]

['Привет', 'мир']

In [99]:
print_simple_data_pipe(datapipe_pairs)

<class 'torch.utils.data.datapipes.iter.combining.ZipperIterDataPipe'>
("This new development in Harry's character may be a disappointment to those readers who enjoyed his old vindictive ways, but it also reinforces the position of pro-Potter people who do not see beneath the surface appearance of the characters and plots.", 'Такое развитие характера Гарри может разочаровать читателей, полюбивших его былую мстительность, но с другой стороны это преображение укрепляет позицию тех, кто не видит глубже сюжета и изображения героев.')
('A nondisclosure clause in the final settlement (the band is back on Elektra) prevents Ulrich, an irrepressible motormouth, from providing any juicy contractual details.', 'Решение суда (группа вернулась под крыло к Elektra Entertainment) предотвратило дальнейшие нападки со стороны неугомонного Ульриха и не позволило ему обнародовать детали нового контракта.')
("When you're 18 or 19 years old, you have that gang mentality in your band.", 'Когда тебе 18 или 19

In [113]:
def yield_tokens_for_pairs(data_iter: torchdata.datapipes.iter.IterDataPipe):
  for eng, rus in data_iter:
    yield eng_tokenizer(eng), [i.text for i in rus_tokenizer(rus)]


def yield_tokens_eng(data_iter: torchdata.datapipes.iter.IterDataPipe):
  for i in data_iter:
    yield eng_tokenizer(i)


def yield_tokens_rus(data_iter: torchdata.datapipes.iter.IterDataPipe):
  for i in data_iter:
    yield [j.text for j in rus_tokenizer(i)]


In [106]:
eng_vocab = torchtext.vocab.build_vocab_from_iterator(
    yield_tokens_eng(datapipe_en),
    min_freq=2,
    specials=['<UNK>', '<SOS>', '<EOS>'],
    special_first=True
)
eng_vocab.set_default_index(eng_vocab['<UNK>'])


In [114]:
rus_vocab = torchtext.vocab.build_vocab_from_iterator(
    yield_tokens_rus(datapipe_rus),
    min_freq=2,
    specials=['<UNK>', '<SOS>', '<EOS>'],
    special_first=True
)
rus_vocab.set_default_index(rus_vocab['<UNK>'])
